In [1]:
import pandas as pd
import numpy as np
from llms import gemini
from llms import chatGPT
import re
import time
from lemmatize import lemmatize
x_chat = chatGPT()
x_gemini = gemini()

In [9]:
df = pd.read_csv("../output/data.csv")
df = df[df["num employees"]=="1-10"]
# df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>'11/30/2022']
# sample_df = df.sample(n=20, random_state=1)
# sample_df = df
# sample_df = sample_df.drop_duplicates(subset=['organization name'], keep='first')
# sample_df = sample_df.reset_index(drop=True)
# sample_df.head()
len(df)

3414

In [6]:
#See format of ONET Task that startup automation jobs will be mapped to
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
df_exp.head()
df_exp.description.iloc[0]

'Sales managers that resolve customer complaints regarding sales and service.'

In [674]:
generated_description_prompt = """
Your role is to describe $company ($website)'s product. 
Then, provide a confidence interval on scale on 1-10 on how sure you are about the response. Please be reasonable.

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

EXAMPLES BELOW
************************
Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo to generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution  providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.
************************

YOUR TURN:
Company: $company
Wesbite: $website
Current Description: $description
________________________
Description (two sentences):
Confidence Interval:
Reasoning:
________________________

"""



In [675]:
parsed_description_prompt = """
Your role is to describe what jobs/tasks, industries, and customer that $company is targeting. Then, provide a confidence interval (1-10) on scale on 1-10 on how sure you are about the response. Please be reasonable.

A response should include:
- tasks/jobs being automated
- the industry that the startup applies to
- specific customers using the tool. DO NOT INCLUDE AN EXPLANATION

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

EXAMPLES BELOW
************************
Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Vetinarians, Pet Businesses

Company: Thunder
Wesbite: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks 
Industry: Cloud Computing
Customers: Developers, GPU Owners

Comapny: NonprofitsHQ
Websit: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit organizations
************************

YOUR TURN:
________________________
Company: $company
Wesbite: $website
Description: $description
________________________
Tasks/Jobs (comma separated list of 4, short):
Industry (1 item):
Customers (comma separated list): 
________________________
Confidence Interval:
Reasoning:
________________________
"""


In [676]:
examples_prompt = """
Your role is to provide 3 two-sentence examples of how the product from $company might be used. Do not mention the name of the company in the examples, and keep the descriptions broad.

Each example should include:
- A detailed description of the job that the tool automates and who performs that job and when.
- The ONET job being automated (preferably one from the database https://www.onetonline.org/) and the ONET task that the tool replaces (preferably one that from https://www.onetcenter.org/dictionary/20.1/excel/task_statements.html)
- A confidence interval (1-10) indicating how sure you are about the accuracy of your response.

PLEASE FOLLOW THE FORMAT EXACTLY FROM THE EXAMPLES!!!!

The goal is to map each example back to ONET jobs. If the job being automated is recognized by ONET, please use the ONET job title. 
If it is not typically found in ONET, use best judgement!

EXAMPLES BELOW: 
************************

Company: HireGuardian
Website: www.hireguardian.com
HireGuardian is a comprehensive workforce management solution that combines employee feedback, background checks, and AI-powered insights to help businesses make informed hiring decisions and build a stronger workforce.  This platform empowers organizations to effectively evaluate potential hires and gain valuable feedback from departing employees, optimizing talent acquisition and retention. 
Tasks/Jobs: Background checks, Employee feedback analysis, Talent acquisition, Workforce management
Industry: Human Resources
People Using Tool: Recruiters, HR professionals 
________________
Example 1: A recruiter uses the platform to automate the process of screening candidates by analyzing their resumes and social media profiles, identifying potential red flags and recommending qualified individuals for further consideration.
ONET JOB automated 1: Human Resource Managers that screen candidates by analyzing their resumes and social media profiles. 
ONET JOB 1: Human Resource Managers
________________
Example 2: An HR manager uses the platform to conduct exit interviews with departing employees, gathering valuable insights into reasons for leaving and areas for improvement within the company. This feedback helps inform future hiring and retention strategies.
ONET JOB automated 2: Human Resource Managers that conduct manual exit interviews and analyzes responses to identify trends. 
ONET JOB 2: Human Resource Managers
________________
Example 3: A company utilizes the platform to monitor employee sentiment and identify potential issues before they escalate into larger problems. By analyzing feedback and identifying patterns, they can proactively address concerns and promote a positive work environment.
ONET JOB automated 3: Human Resource Managers that monitor employee sentiment and identifies potential issues manually. 
ONET JOB 3: Human Resource Managers
________________

Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
________________
Example 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
ONET JOB automated 1: Solar Photovoltaic Installers that plan, design, and visualize the installation of a rooftop solar panel.
ONET JOB 1: Solar Photovoltaic Installers
________________
Example 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
ONET JOB automated 2:  Solar Photovoltaic Installers that research financing options for solar panel installation.
ONET JOB 2: Solar Photovoltaic Installers
________________
Example 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
ONET JOB automated 3:  A solar installer manages the installation of the solar panels, including tracking materials, scheduling technicians, and coordinating with subcontractors.
ONET JOB 3: Solar Energy Installation Managers
________________
************************

YOUR TURN:
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
Example 1: 
ONET JOB automated 1:
ONET JOB 1: 
Confidence Interval 1:
Reasoning 1:
________________
Example 2: 
ONET JOB automated 2:
ONET JOB 2: 
Confidence Interval 2:
Reasoning 2:
________________
Example 3:
ONET JOB automated 3:
ONET JOB 3: 
Confidence Interval 3:
Reasoning 3:
________________
"""


In [755]:
class prompting():
    def __init__(self):
        pass
    def iterate(self, df, prompt_template, args, value, start=0, end=False):
        if end == False:
            end = len(df)

        self.results_df = pd.DataFrame(columns=["organization name",value])
        
        if value in list(df.columns): 
            if start != 0:
                self.results_df = pd.concat([df[["organization name",value]].iloc[:start],self.results_df],axis=0)
            df = df.drop(columns=[value])
        
        
        for i, row in list(df.iterrows())[start:end]:
            failure_count = 0
            prompt = prompt_template
            while True:
                try:
                    name = row['organization name']
                    website = row['website']
                    print(f"******************************\nProcessing {i}: {name}, {website}")
                    if "generated_description" in df.columns:
                        print(row["generated_description"])

                    for arg in args: 
                        prompt = prompt.replace(f"${arg[0]}", row[arg[1]])
                    result = x_gemini.ask(prompt)
                    if result == "N/A": break #explicit material
 
                    text = re.sub(r"#|#\s+|_|\*", "", result).strip()
                    print(f"Result:\n{text}\n")
   
                    self.results_df.loc[i] = [name, result]
                    break 
                
                except Exception as e:
                    print(failure_count)
                    failure_count += 1
                    if failure_count > 10: 
                        break
                    print(f"Error processing {index}, {row['organization name']}: {e}")
                    time.sleep(20)  



        df = df.merge(self.results_df, on='organization name', how='left')
        return df

prompting_class = prompting()


In [720]:
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]

def extract_data(text):
    if pd.isnull(text): return {}
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace(" (two sentences)","").replace("/10","").replace("_","").replace("#","")
    pattern = r"^.*?\s*Description:\s*(.*)Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        print("FAILURE TO MATCH")
        return {}
        
args = [["company","organization name"],["website","website"], ["description","description_all"]]

sample_df = prompting_class.iterate(sample_df, generated_description_prompt, args, "generated_description_llm",898)

for col in cols:
    if col in list(sample_df.columns): sample_df=sample_df.drop(columns=[col])
        
results_df = pd.DataFrame(list(sample_df.apply(lambda x: extract_data(x["generated_description_llm"]),axis=1)))
sample_df = pd.concat([sample_df, results_df],axis=1)

sample_df.to_csv('../output/df_with_generated_description.csv', index=False)

******************************
Processing 898: Fincortex.ai, fincortex.ai
Result:
Description (two sentences): Fincortex.ai provides an AI-powered virtual assistant named Leo that allows users to interact with their business data through voice commands. Leo can understand complex commands and questions,  provide insights, automate workflows, and complete tasks. 
Confidence Interval: 8/10
Reasoning: This description is based on the website's language and the information provided about Leo's capabilities. While the website does not specify specific functionalities, it emphasizes the conversational nature of the product and its ability to provide insights and automation.

******************************
Processing 899: Peregrine Foundry, —
Result:
Description (two sentences): Peregrine Foundry provides an AI-powered platform designed to automate various workflows and business processes. It aims to streamline operations by enabling intelligent automation, reducing manual effort, and improvi

AttributeError: 'float' object has no attribute 'replace'

In [736]:
# Restart if cut in the middle of run!
# sample_df = sample_df.merge(prompting_class.results_df, on='organization name', how='left')

In [727]:
sample_df = sample_df.dropna(subset=["generated_description"],axis=0)
sample_df = sample_df.drop_duplicates(subset=['organization name'], keep='first')
sample_df = sample_df.reset_index(drop=True)

In [728]:
len(sample_df)

992

In [741]:
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]
    
args = [["company","organization name"],["website","website"], ["description","generated_description"]]

sample_df = prompting_class.iterate(sample_df, parsed_description_prompt, args, "parsed_description_llm",327)

sample_df.to_csv('../output/df_with_parsed_description.csv', index=False)

******************************
Processing 327: DRInclusive, —
DRInclusive is a medical device company developing a GPS-Enabled AI-Powered Health Monitoring Solution integrated into wheelchair seating. Their technology aims to improve the health and independence of wheelchair users by providing real-time health data and enabling quicker response in emergencies.
Result:
Tasks/Jobs: Health data monitoring, Emergency response, Mobility tracking, Posture analysis
Industry: Healthcare Technology
Customers: Wheelchair users, Healthcare providers 

Confidence Interval: 8
Reasoning: The description clearly states the technology's focus on health monitoring and emergency response, which directly translates to these tasks. The industry is also evident, being a medical device company. While the specific customer base is not explicitly stated, it's reasonable to assume wheelchair users and their healthcare providers would be the primary targets.

******************************
Processing 328: Prakt

In [754]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text)  
    text = text.replace("_", "").replace("*", "").replace("#", "")
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(.*?)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": tasks_jobs + "\n" + industry + "\n" + customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(sample_df.columns): sample_df=sample_df.drop(columns=[col])
        
results_df = pd.DataFrame(list(sample_df.apply(lambda x: extract_data(x["parsed_description_llm"]),axis=1)))
sample_df = pd.concat([sample_df, results_df],axis=1)


In [756]:
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_reasoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_reasoning","situation3", "situation3_conf_interval", "situation3_conf_interval_reasoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']

args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]
sample_df = prompting_class.iterate(sample_df, examples_prompt, args, "examples_llm")

sample_df.to_csv('../output/df_with_examples.csv', index=False)

******************************
Processing 0: Catio, San Francisco Bay Area, Silicon Valley, West Coast
Catio is a cloud architecture platform that helps companies design and optimize their cloud-native tech stacks. Using AI, Catio provides guidance on technology selection, planning, and continuous improvement, acting as a "copilot" for architects and engineers.
Result:
Catio Examples:

Example 1: A cloud architect uses Catio to evaluate different cloud service offerings from various providers, comparing costs, performance, and security features to select the most suitable options for their company's specific needs. 

ONET JOB automated 1:  Computer and Information Systems Managers that evaluate cloud service offerings from different providers to select the most suitable options for the company's needs.
ONET JOB 1: Computer and Information Systems Managers
Confidence Interval 1: 9
Reasoning 1: The provided description aligns with the responsibilities of Computer and Information Systems 

In [759]:

def extract_data(text):
    if pd.isnull(text): return {}
    # Normalize the text to ensure consistent whitespace and remove unwanted characters.
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("*", "").replace("/10", "").replace("_", "").replace("#", "")
    vals = {}
    i = 1
    while i <=3:
        # Regex pattern adjusted to handle fractions in confidence intervals like '9/10'
        pattern = rf"Example\s+{i}:\s+(.*?)\s+ONET JOB automated\s+{i}:\s+(.*?)\s+ONET JOB\s+{i}:\s+(.*?)\s+Confidence Interval\s+{i}:\s+(\d+(?:/\d+)?)\s+Reasoning\s+{i}:\s+(.*?)(?=\s*Example\s+{i + 1}:|$)"
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            print(text)
            print(f"No matches found for Example {i}")  # Debug if no examples are found
            break

        example_text, onet_job_automated, onet_job, confidence_interval, reasoning = match.groups()
        example_key = f"Example{i}"
        vals[example_key] = example_text.strip()
        vals[f"Job{i}"] = onet_job_automated.strip()
        vals[f"Job{i}_title"] = onet_job.strip()
        vals[f"situation{i}_conf_interval"] = confidence_interval.strip()
        vals[f"situation{i}_conf_interval_reasoning"] = reasoning.strip()

        i += 1  # Prepare to search for the next example
    return vals

for col in cols:
    if col in list(sample_df.columns): sample_df=sample_df.drop(columns=[col])

results_df = pd.DataFrame(list(sample_df.apply(lambda x: extract_data(x["examples_llm"]),axis=1)))
sample_df = pd.concat([sample_df, results_df],axis=1)


 KeyAI Use Case Examples: Example 1: An individual who has lost their private key to their cryptocurrency wallet uses KeyAI's technology to recover their lost crypto assets. The platform analyzes the blockchain data and applies advanced algorithms to solve the discrete logarithm problem, allowing them to regain access to their funds. ONET JOB automated 1: Individuals who manually attempt to recover lost cryptocurrency keys, potentially through brute-force methods or utilizing online tools with limited success rates. ONET JOB 1: Not applicable - This is a very niche task not typically found in ONET. Confidence Interval 1: 7 - KeyAI's focus on recovering lost crypto keys aligns with the user's situation, though the task isn't directly covered by a specific ONET job. Reasoning 1: This example is a direct application of KeyAI's service, targeting individuals who have lost their crypto keys. While not a formal ONET job, it's a real-world problem that the tool aims to solve. Example 2: A cry

In [767]:
# sample_df[pd.isnull(sample_df.examples_llm)].values

In [768]:
sample_df = sample_df.dropna(subset=["Example1"],axis=0)
sample_df = sample_df.drop_duplicates(subset=['organization name'], keep='first')
sample_df = sample_df.reset_index(drop=True)

In [770]:
sample_df.to_csv('../output/df_with_examples.csv', index=False)

In [2]:
sample_df = pd.read_csv('../output/df_with_examples.csv')
job_cols = [[f"Job{i}",f"Job{i}_title"] for i in range(1,4)]
for job in job_cols:
    for col in job:
        sample_df[col+"_lemmatize"] = sample_df[col].apply(lambda x: lemmatize(x))

lemmatize_cols = [[f"Job{i}_lemmatize",f"Job{i}_title_lemmatize"] for i in range(1,4)]

In [3]:
sample_df.Job3_lemmatize.iloc[0]

'Systems Administrators monitor analyze cloud environment identify bottleneck recommend adjustment efficiency'

In [ ]:
embedding_cols = []
for i in range(1,4):  
    embedding_cols += [f"Job{i}_embedding",f"Job{i}_title_embedding"]

results = []
for i, x in list(sample_df.iterrows())[0:]:
    name = x["organization name"]
    print(f"Processing {str(i), len(results)}: {name}")
    job_embeddings = [name]
    for job in lemmatize_cols: 
        for col in job:
            if pd.isnull(x[col]):
                job_embeddings.append(np.nan)
                continue
            job_embeddings.append(x_chat.get_embedding(x[col]))
    results.append(job_embeddings)


results_df = pd.DataFrame(results, columns=['organization name']+embedding_cols)
sample_df = sample_df.merge(results_df, on="organization name")

sample_df.to_csv('../output/df_with_examples_embeddings.csv', index=False)

Processing ('0', 0): Catio
Processing ('1', 1): Helios Artificial Intelligence
Processing ('2', 2): Beagle
Processing ('3', 3): Sunrise
Processing ('4', 4): Algorized
Processing ('5', 5): Edifii
Processing ('6', 6): Steno.ai
Processing ('7', 7): Digital Will & Trust
Processing ('8', 8): Jounce
Processing ('9', 9): Vestinda
Processing ('10', 10): SneakPeek
Processing ('11', 11): DivySci
Processing ('12', 12): CTFGuide
Processing ('13', 13): Motivision
Processing ('14', 14): RadUnity
Processing ('15', 15): MedDefend
Processing ('16', 16): Flotation Innovation
Processing ('17', 17): Trust Fund
Processing ('18', 18): PhotoAi
Processing ('19', 19): Augrade
Processing ('20', 20): VideoGen
Processing ('21', 21): ColentAI
Processing ('22', 22): FleetWorks Technology
Processing ('23', 23): Move AI
Processing ('24', 24): Tripsby.AI
Processing ('25', 25): Skin Dossier
Processing ('26', 26): Parsed
Processing ('27', 27): denyify
Processing ('28', 28): Overlap Capital
Processing ('29', 29): OpenEnt

In [605]:
sample_df.head(100)

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,Job3_title,situation3_conf_interval,situation3_conf_interval_reasoning,name,Job1_embedding,Job1_title_embedding,Job2_embedding,Job2_title_embedding,Job3_embedding,Job3_title_embedding
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,Volunteer Coordinators,7,While volunteer coordination is a complex role...,NonprofitsHQ,"[-0.02567981742322445, -0.004415797535330057, ...","[-0.017821840941905975, 0.007271148264408112, ...","[-0.029476625844836235, -0.01559417974203825, ...","[-0.007853823713958263, 0.005379451904445887, ...","[-0.009866348467767239, -0.03487496078014374, ...","[-0.0030880924314260483, -0.011928810738027096..."


In [6]:
results[-1]

['Showspace',
 [-0.00035719526931643486,
  0.046300556510686874,
  -0.020139776170253754,
  -0.01687169075012207,
  0.029493261128664017,
  0.0036202496849000454,
  -0.006624714471399784,
  0.052289362996816635,
  -0.048103637993335724,
  -0.012903301976621151,
  0.03002452664077282,
  -0.01996268890798092,
  -0.012983797118067741,
  -0.055541351437568665,
  0.00011703176278388128,
  0.01827229931950569,
  -0.03153782710433006,
  -0.0031211823225021362,
  -0.03483811020851135,
  -0.049391552805900574,
  0.014730531722307205,
  -0.061530157923698425,
  0.0008431820897385478,
  0.018449388444423676,
  -0.016791196539998055,
  -0.0041817002929747105,
  0.007973001338541508,
  0.000710868916939944,
  0.0012577298330143094,
  0.007836160250008106,
  0.014899570494890213,
  0.022989289835095406,
  0.0018664711387827992,
  0.007135855965316296,
  0.006918520200997591,
  -0.013048192486166954,
  0.027931664139032364,
  0.035932838916778564,
  -0.011937365867197514,
  0.011301456950604916,
  0.